Visualizing the Paypal Mafia's influence using a network graph. Data is from regulatory disclosures: 3,4,5, SC 13D(/A), and SC 13G(/A). 

Code includes preparation as well as plotting. You can skip the preparation stage and go straight to the plotting using dataset data/paypal_edges.csv

![paypal] (plots/paypal_mafia_network_graph.png)

In [1]:
# Getting the CIKs of the PayPal Mafia members
paypal_mafia_cik = {
    "Peter Thiel": "1211060",
    "Max Levchin": "1539853",
    "Elon Musk": "1494730",
    "David O. Sacks": "1891801",
    "Roelof Botha": "1222287",
    "Reid Hoffman": "1519339",
    "Chad Hurley": "1750295",
    "Luke Nosek": "1835310",
    "Keith Rabois": "1539865",
    "Jack Selby": "1870713",
}

In [35]:
# Downloading the data. the final dataset is stored in the data/ folder so you can skip this

from datamule import PremiumDownloader, Downloader, Portfolio


downloader = PremiumDownloader()
# downloader = Downloader() # This works, but is slower.

ciks = [key for key in paypal_mafia_cik.values()]

# Note: sometimes D/A and G/A can be submitted even if there is no D or G.
downloader.download_submissions(submission_type=['3','4','5','SC 13D','SC 13G','SC 13D/A','SC 13G/A'],cik=ciks,output_dir='ingest/paypal')


Cost: $0.004670000000 downloads + $0.005699293000 row reads = $0.010369293000
Balance: $9.594538817000


Processing files: 100%|██████████| 467/467 [00:06<00:00, 77.20it/s] 


Processing completed in 6.05 seconds


In [ ]:
# Constructing the edge list
import pandas as pd

portfolio = Portfolio('ingest/paypal')

edge_list = []
for submission in portfolio:
    metadata = submission.metadata['submission']
    filing_date = metadata['FILING-DATE']

    # Try block is here because the parse SGML function I wrote to parse SGML fails on certain edge cases. I'm tired and don't want to mess with pointers and mutable objects right now. Will fix later.
    try:
        # 3,4,5 metadata (Forms 3, 4, 5)
        if 'REPORTING-OWNER' in metadata.keys():
            # Create edges between each reporting owner and the issuer
            issuer_data = metadata['ISSUER'][0]  # Assuming single issuer
            issuer_name = issuer_data['COMPANY-DATA']['CONFORMED-NAME']
            issuer_cik = issuer_data['COMPANY-DATA']['CIK']
            
            for owner in metadata['REPORTING-OWNER']:
                owner_name = owner['OWNER-DATA']['CONFORMED-NAME']
                owner_cik = owner['OWNER-DATA']['CIK']
                
                edge_list.append({
                    'date': filing_date,
                    'source_name': owner_name,
                    'source_cik': owner_cik,
                    'target_name': issuer_name,
                    'target_cik': issuer_cik,
                })

        # SC 13D(/A) and SC 13G(/A) metadata
        else:
            # Archive style metadata
            if 'SUBJECT-COMPANY' in metadata.keys():
                subject_name = metadata['SUBJECT-COMPANY']['COMPANY-DATA']['CONFORMED-NAME']
                subject_cik = metadata['SUBJECT-COMPANY']['COMPANY-DATA']['CIK']
                filer_name = metadata['FILED-BY']['COMPANY-DATA']['CONFORMED-NAME']
                filer_cik = metadata['FILED-BY']['COMPANY-DATA']['CIK']

            # Header style metadata
            else:
                subject_name = metadata['SUBJECT COMPANY']['COMPANY DATA']['COMPANY CONFORMED NAME']
                subject_cik = metadata['SUBJECT COMPANY']['COMPANY DATA']['CENTRAL INDEX KEY']
                filer_name = metadata['FILED BY']['COMPANY DATA']['COMPANY CONFORMED-NAME']
                filer_cik = metadata['FILED BY']['COMPANY DATA']['CENTRAL INDEX KEY']

            edge_list.append({
                'date': filing_date,
                'source_name': filer_name,
                'source_cik': filer_cik,
                'target_name': subject_name,
                'target_cik': subject_cik,
            })

    except Exception as e:
        print(f"If you see this error after January 10th, feel free to shame me for my laziness. {submission.path}: {str(e)}")

# Convert to DataFrame
edges_df = pd.DataFrame(edge_list)
# convert ciks to int
edges_df['source_cik'] = edges_df['source_cik'].astype(int)
edges_df['target_cik'] = edges_df['target_cik'].astype(int)
# 

edges_df.to_csv('data/paypal_edges.csv', index=False)

Loading 467 submissions


Loading submissions: 100%|██████████| 467/467 [00:00<00:00, 2130.33it/s]

If you see this error after January 10th, feel free to shame me for my laziness. ingest\paypal\000095017024009914: 'OWNER-DATA'
If you see this error after January 10th, feel free to shame me for my laziness. ingest\paypal\000095017024012830: 'OWNER-DATA'


In [45]:
# Plotting the network graph

import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
import matplotlib.patches as mpatches

# Create directed graph
G = nx.DiGraph()

# Add edges and store names as node attributes
for _, row in edges_df.iterrows():
    # Extract last name for mafia members
    source_name = row['source_name'].split()[0] if str(row['source_cik']) in paypal_mafia_cik.values() else row['source_name']
    G.add_node(str(row['source_cik']), name=source_name)
    G.add_node(str(row['target_cik']), name=row['target_name'])
    G.add_edge(str(row['source_cik']), str(row['target_cik']))

# Set style
plt.style.use('dark_background')
fig = plt.figure(figsize=(20, 11.25), dpi=300, facecolor='#0D1117')
ax = plt.gca()
ax.set_facecolor('#0D1117')

# Use circular layout
pos = nx.circular_layout(G, scale=0.7)

# Define Thiel's color and connection color
thiel_cik = "1211060"
thiel_color = "#FF6B6B"  # coral red
thiel_connection_color = lighten_color(thiel_color)
mafia_color = "#4ECDC4"  # turquoise for other mafia members

# Create color map for nodes
node_colors = []
for node in G.nodes():
    node_str = str(node)
    if node_str == thiel_cik:  # Thiel
        node_colors.append(thiel_color)
    elif node_str in paypal_mafia_cik.values():  # Other PayPal Mafia members
        node_colors.append(mafia_color)
    else:
        # Check if node is connected to Thiel
        if G.has_edge(thiel_cik, node_str):
            node_colors.append(thiel_connection_color)
        else:
            node_colors.append('#808080')  # gray for others

# Draw nodes with a glowing effect
nx.draw_networkx_nodes(G, pos, node_size=700, node_color=node_colors, alpha=0.9,
                      edgecolors='white', linewidths=1)

# Draw edges with a subtle curve and glow
nx.draw_networkx_edges(G, pos, edge_color='#ffffff33', arrows=True, 
                      width=1, alpha=0.3, connectionstyle="arc3,rad=0.2")

# Create labels
labels = {node: G.nodes[node]['name'] for node in G.nodes()}
label_pos = {node: (coord[0]*1.05, coord[1]*1.05) for node, coord in pos.items()}

# Draw labels with semi-transparent background
for node, (x, y) in label_pos.items():
    bbox_color = '#1F2937'
    plt.text(x, y, labels[node], 
             fontsize=7,
             color='white',
             bbox=dict(facecolor=bbox_color, edgecolor='none', alpha=0.7, pad=2),
             horizontalalignment='center',
             verticalalignment='center',
             fontweight='bold')

# Add legend
legend_elements = [
    mpatches.Patch(color=thiel_color, label='Peter Thiel'),
    mpatches.Patch(color=thiel_connection_color, label='Thiel Connections'),
    mpatches.Patch(color=mafia_color, label='Other PayPal Mafia'),
    mpatches.Patch(color='#808080', label='Other Entities')
]
plt.legend(handles=legend_elements, 
          loc='upper right', 
          bbox_to_anchor=(1, 1),
          facecolor='#1F2937',
          edgecolor='none',
          labelcolor='white')

plt.title("PayPal Mafia: Network of Insider Disclosures\n", 
          color='white', 
          pad=20, 
          fontsize=16,
          fontweight='bold')

# Add subtitle
plt.figtext(0.5, 0.95, 
            "Generated from SEC Forms 3, 4, 5 and Schedules 13D/A, 13G/A", 
            color='#666666',
            fontsize=10,
            ha='center')

plt.figtext(0.99, 0.01, 'Data source: SEC EDGAR', 
            ha='right', va='bottom', color='#666666', fontsize=8)

plt.axis('off')
plt.tight_layout()

plt.savefig('plots/paypal_mafia_network_graph.png', 
            dpi=300,
            bbox_inches='tight',
            pad_inches=0.5,
            facecolor='#0D1117',
            edgecolor='none')

plt.close()
